# Testing VPU & Signal Reconstruction

The aim of this blog post is to test that we can correctly reconstruct a simple signal. And also to better understand the data of each stage.

## Test Binary Vectors
Let's test first by randomly selecting between [0, 0] and [1, 1] vectors.

In [1]:
from src.var_processor.vpu import VPU, VPUBinary
import random
import numpy as np

In [2]:
random.random()

0.46799349226432885

In [3]:
def rand_same():
    a = np.empty([2, 1])
    a.fill(np.random.randint(2))
    return a

In [4]:
def rand_diff():
    a = np.zeros([2, 1])
    index = np.random.randint(2)
    a[index] = 1
    return a

In [5]:
print(rand_same(), rand_same().shape)

[[1.]
 [1.]] (2, 1)


In [6]:
print(rand_diff(), rand_diff().shape)

[[1.]
 [0.]] (2, 1)


In [7]:
vpu = VPU(2)
print(vpu.cu.covariance, vpu.pi.ev)
for _ in range(0, 1000):
    vpu.update_cov(rand_same())
    vpu.pi.iterate(cov=vpu.cu.covariance)
print(vpu.cu.covariance, vpu.pi.ev)

[[0. 0.]
 [0. 0.]] [[0.81341381]
 [0.58168546]]
[[0.24586054 0.24586054]
 [0.24586054 0.24586054]] [[0.70710678]
 [0.70710678]]


In [8]:
print(vpu.pi.eigenvalue, vpu.pi.eigenvector)

[[0.49172109]] [[0.70710678]
 [0.70710678]]


In [9]:
np.sqrt(vpu.pi.eigenvalue)

array([[0.70122827]])

In [10]:
np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))

array([[0.49584326],
       [0.49584326]])

Does the above expression thus give you something to do with the probability for the vector?

In [11]:
np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean

array([[0.96384326],
       [0.96384326]])

We can use this to same with x = 1 or 0? Or x = -1, or 1?

In [12]:
print(
    np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean,
    -1*np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean
)

[[0.96384326]
 [0.96384326]] [[-0.02784326]
 [-0.02784326]]


Yes - x = -1 or 1 or random value within this range.

In [26]:
# Adapt to test

vpu = VPU(2)
for _ in range(0, 1000):
    vpu.update_cov(rand_same())
    vpu.pi.iterate(cov=vpu.cu.covariance)
# Check all values of covariance matrix are the same
print(vpu.cu.covariance, vpu.cu.covariance[0])
assert np.allclose(vpu.cu.covariance, vpu.cu.covariance[0])
# Check eigenvector has values of root 2
print(vpu.pi.eigenvector, 1/np.sqrt(2))
assert np.allclose(vpu.pi.eigenvector, 1/np.sqrt(2))
sample_1 = np.dot(
        vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean
sample_minus1 = -1*np.dot(
        vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean
assert np.allclose(sample_1, np.array([1, 1]), rtol=0.05, atol=0.05)
assert np.allclose(sample_minus1, np.array([0, 0]), rtol=0.05, atol=0.05)

[[0.24691728 0.24691728]
 [0.24691728 0.24691728]] [0.24691728 0.24691728]
[[0.70710678]
 [0.70710678]] 0.7071067811865475


In [24]:
1/np.sqrt(2)

0.7071067811865475

In [13]:
vpu = VPU(2)
print(vpu.cu.covariance, vpu.pi.ev, sep="\n", end="\n\n")
for _ in range(0, 1000):
    vpu.update_cov(rand_diff())
    vpu.pi.iterate(cov=vpu.cu.covariance)
print(vpu.cu.covariance, vpu.pi.ev, vpu.pi.eigenvalue, sep="\n", end="\n\n")
print(
    vpu.cu.mean,
    np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue)), 
    np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean,
    sep="\n",
    end="\n\n"
)

[[0. 0.]
 [0. 0.]]
[[0.79083108]
 [0.61203448]]

[[ 0.24667776 -0.24667776]
 [-0.24667776  0.24667776]]
[[ 0.70710678]
 [-0.70710678]]
[[0.49335551]]

[[0.486]
 [0.514]]
[[ 0.49666665]
 [-0.49666665]]
[[0.98266665]
 [0.01733335]]



How would we get a sample of [0, 1] if x = 0 then we just have mean. Ah - x needs to be between -1 and 1.

In [14]:
# Forward Pass
a = rand_diff()
print(a, np.dot(vpu.pi.eigenvector.T, a))

[[0.]
 [1.]] [[-0.70710678]]


In [15]:
np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean

array([[0.98266665],
       [0.01733335]])

In [16]:
-1*np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean

array([[-0.01066665],
       [ 1.01066665]])

In [34]:
# Adapt to test

vpu = VPU(2)
for _ in range(0, 1000):
    vpu.update_cov(rand_diff())
    vpu.pi.iterate(cov=vpu.cu.covariance)
# Check diagonal values of covariance matrix are the same
# Use https://docs.scipy.org/doc/numpy/reference/generated/numpy.diagonal.html
print(vpu.cu.covariance[0], -1*vpu.cu.covariance[-1])
assert np.allclose(vpu.cu.covariance[0], -1*vpu.cu.covariance[-1])
# Check eigenvector has values of root 2
print(vpu.pi.eigenvector, 1/np.sqrt(2))
assert np.allclose(np.abs(vpu.pi.eigenvector), 1/np.sqrt(2))
# Check different signs
assert np.allclose(vpu.pi.eigenvector[0], -1*vpu.pi.eigenvector[1])
sample_1 = np.dot(
        vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean
sample_minus1 = -1*np.dot(
        vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean
print(sample_1, np.flipud(sample_minus1))
assert np.allclose(sample_1, np.flipud(sample_minus1), rtol=0.1, atol=0.1)

[ 0.24607419 -0.24607419] [ 0.24607419 -0.24607419]
[[ 0.70710678]
 [-0.70710678]] 0.7071067811865475
[[ 1.02705866]
 [-0.02705866]] [[0.96505866]
 [0.03494134]]


## Test Reconstruction

So we do a forward pass - get the output r, reconstruct then subtract.

In [35]:
# Setup VPU
vpu = VPU(2)
for _ in range(0, 1000):
    vpu.update_cov(rand_same())
    vpu.pi.iterate(cov=vpu.cu.covariance)

In [40]:
# Test forward pass with no processing
ran = rand_same()
r = vpu.forward(ran)
print(ran, r)

[[1.]
 [1.]] [[1.41421356]]


So [0, 0] r = 0 and for [1, 1] r = 1.414.

In [41]:
vpu.cu.mean

array([[0.509],
       [0.509]])

In [68]:
# Test forward pass with mean removal
ran = rand_same()
r = vpu.forward(ran-vpu.cu.mean)
print(ran, r)

[[0.]
 [0.]] [[-0.7198347]]


Now we have [0, 0] = -0.7198, [1, 1] = 0.69437886.

In [52]:
print(vpu.pi.eigenvector, vpu.pi.eigenvalue)
vpu.pi.eigenvector / np.sqrt(vpu.pi.eigenvalue)

[[0.70710678]
 [0.70710678]] [[0.49364388]]


array([[1.00641736],
       [1.00641736]])

In [59]:
# Test forward pass with mean removal and divide by eigenvalue - SAME
ran = rand_same()
r = vpu.forward((ran-vpu.cu.mean)/np.sqrt(vpu.pi.eigenvalue))
print(ran, r)

[[0.]
 [0.]] [[-1.02453288]]


Now we have outputs between -1 and 1.

In [67]:
# Test forward pass with mean removal and divide by eigenvalue - DIFFERENCES
ran = rand_diff()
r = vpu.forward((ran-vpu.cu.mean)/np.sqrt(vpu.pi.eigenvalue))
print(ran, r)

[[0.]
 [1.]] [[-0.01811551]]


Non-feature outputs are then 0.

In [159]:
vpu.cu.mean

array([[0.509],
       [0.509]])

### Test Signal Reconstruction Over Time

Let's say we have a random 8-bit static signal - [28, 234] - let's experiment with putting this through the system and looking at the output over 255 time steps.

In [115]:
data_in = np.random.randint(255, size=(2, 1)); print(data_in)

[[19]
 [76]]


In [116]:
# Determine signal mean - we'll do a test version
signal_mean = np.random.randint(255, size=(2, 255)).mean(axis=1).reshape(2,1); print(signal_mean)

[[128.99215686]
 [129.31764706]]


In [117]:
# Subtract mean and binary threshold +ve and -ve
zero_mean = data_in - signal_mean; print(zero_mean)

[[-109.99215686]
 [ -53.31764706]]


To cope with the sign we can just - get an array indicating the sign, threshold the absolute, then re-apply the sign.

Use - https://docs.scipy.org/doc/numpy/reference/generated/numpy.sign.html.

But we need to know the range for determining - does this require symmetric ranges either side of the mean? It will be 255-min_mean

In [118]:
signs = np.sign(zero_mean); print(signs)

[[-1.]
 [-1.]]


In [119]:
pbt_range = int(255-signal_mean.min()); print(pbt_range)

126


In [120]:
rand_vals = np.random.randint(pbt_range, size=data_in.shape)
binary_values = np.where(np.abs(zero_mean) > rand_vals, 1, 0)
print(binary_values)

[[1]
 [1]]


In [151]:
# Turn into a short function
def signal_pre_processor(signal, mean):
    """Remove mean and convert to range {-1, 0, 1}"""
    zero_mean = signal - mean
    signs = np.sign(zero_mean)
    rand_vals = np.random.uniform(size=zero_mean.shape)*mean
    binary_values = np.where(np.abs(zero_mean) > rand_vals, 1, 0)
    return binary_values*signs

In [155]:
signal_pre_processor(data_in, signal_mean)

array([[-1.],
       [-0.]])

In [131]:
summed = signal_pre_processor(data_in, signal_mean)
for _ in range(0, int(signal_mean.min())):
    summed += signal_pre_processor(data_in, signal_mean)
print(summed+signal_mean)

[[18.99215686]
 [70.31764706]]


Why are we getting 225 and 80? Something to do with the pbt_range? No - we need to make sure we are summing over the signal mean and then adding the mean.

This can likely be a scalar mean. But if our signals are all different? 

Need to evaluate each element in multiples of the mean value.

We can add this to the sensor routine? DONE

In [135]:
print(255 // signal_mean, 255 % signal_mean, sep="\n")

[[1.]
 [1.]]
[[126.00784314]
 [125.68235294]]


In [136]:
print(3467 // signal_mean, 3467 % signal_mean, sep="\n")

[[26.]
 [26.]]
[[113.20392157]
 [104.74117647]]


In [137]:
summed = signal_pre_processor(data_in, signal_mean)
for _ in range(0, 3467):
    summed += signal_pre_processor(data_in, signal_mean)
print(summed)

[[-3018.]
 [-1495.]]


In [143]:
(summed / (3467 / signal_mean))+signal_mean

array([[16.70535865],
       [73.55477358]])

So we just divide the sum by the (total count / signal mean).

To determine we can generate a random number between 0 and 1 and times by the signal mean.

In [150]:
rand_vals = np.random.uniform(size=zero_mean.shape)*signal_mean
binary_values = np.where(np.abs(zero_mean) > rand_vals, 1, 0)
print(zero_mean, rand_vals, binary_values, sep="\n")

[[-109.99215686]
 [ -53.31764706]]
[[ 66.18549605]
 [125.70555556]]
[[1]
 [0]]


The above is now added to sensor. Anyway back to reconstruction.

# Reconstructing a Static Signal

* So we generate a random data_in vector. 
* Remove the mean and PBT as above. 
* Then provide as upper input to first VPU. 
* Have alpha = 1 and have the residual as the same input. 
* Repeat 1000 times. 
* Then loop output r in a feedback run (1000 times). 
* Get prediction and compute residual. 
* Repeat with another VPU and residual signal. 
* Sum first residual and second residual over time and compare with original random vector.

In [165]:
data_in = np.random.randint(255, size=(2, 1)); print(data_in)

[[ 61]
 [196]]


In [166]:
mean_sum = 0
count = 0
mean_sum += data_in
count += 1
mean = mean_sum/count
ternary_input = signal_pre_processor(data_in, mean); print(mean, ternary_input, sep="\n")

[[ 61.]
 [196.]]
[[0.]
 [0.]]


In [195]:
# Initialise the two VPUs
vpu_1 = VPU(2)
vpu_2 = VPU(2)

Ah we can't compute the mean as we only have a static value! Let's just hard code this as [128, 128]

In [196]:
mean = np.asarray([128, 128]).reshape(-1, 1)

In [201]:
# Loop 
for _ in range(0, 1000):
    ternary_input = signal_pre_processor(data_in, mean)
    vpu.update_cov(ternary_input)
    vpu.pi.iterate(cov=vpu.cu.covariance)
    r = vpu.forward(ternary_input)
    pred = vpu.backward(r) 
    # Check values
    print(
        f"PBT: {ternary_input.T}", 
        f"r: {r}", 
        f"ev: {vpu.pi.eigenvector.T}",
        f"lambda: {vpu.pi.eigenvalue}",
        f"Scaled r: {r/np.sqrt(vpu.pi.eigenvalue)}", 
        f"Rounded r: {np.round(r)}", 
        f"Predictions: {pred.T}", 
        f"Rounded preds: {np.round(pred).T}", 
        sep="\n", end="\n-------\n")

PBT: [[-1  0]]
r: [[-0.96267003]]
ev: [[ 0.96267003 -0.2706777 ]]
lambda: [[0.30303478]]
Scaled r: [[-1.74876404]]
Rounded r: [[-1.]]
Predictions: [[-0.92673358  0.26057331]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[-1  0]]
r: [[-0.96271254]]
ev: [[ 0.96271254 -0.27052644]]
lambda: [[0.30303124]]
Scaled r: [[-1.74885147]]
Rounded r: [[-1.]]
Predictions: [[-0.92681544  0.2604392 ]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[-1  0]]
r: [[-0.96276684]]
ev: [[ 0.96276684 -0.27033314]]
lambda: [[0.30302772]]
Scaled r: [[-1.74896029]]
Rounded r: [[-1.]]
Predictions: [[-0.92691999  0.26026779]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96278716 -0.27026077]]
lambda: [[0.30302288]]
Scaled r: [[0.]]
Rounded r: [[0.]]
Predictions: [[0. 0.]]
Rounded preds: [[0. 0.]]
-------
PBT: [[0 1]]
r: [[-0.27017173]]
ev: [[ 0.96281215 -0.27017173]]
lambda: [[0.3030074]]
Scaled r: [[-0.49080987]]
Rounded r: [[-0.]]
Predictions: [[-0.26012463  0.07299276]]
Rounded preds: [[-0.  0.]]
--

PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96231516 -0.27193664]]
lambda: [[0.30296248]]
Scaled r: [[0.]]
Rounded r: [[0.]]
Predictions: [[0. 0.]]
Rounded preds: [[0. 0.]]
-------
PBT: [[-1  1]]
r: [[-1.23430748]]
ev: [[ 0.96229322 -0.27201425]]
lambda: [[0.30297876]]
Scaled r: [[-2.2424216]]
Rounded r: [[-1.]]
Predictions: [[-1.18776572  0.33574923]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[0 1]]
r: [[-0.27202059]]
ev: [[ 0.96229143 -0.27202059]]
lambda: [[0.30296339]]
Scaled r: [[-0.4942045]]
Rounded r: [[-0.]]
Predictions: [[-0.26176308  0.0739952 ]]
Rounded preds: [[-0.  0.]]
-------
PBT: [[-1  1]]
r: [[-1.23438285]]
ev: [[ 0.96226351 -0.27211934]]
lambda: [[0.30297967]]
Scaled r: [[-2.24255516]]
Rounded r: [[-1.]]
Predictions: [[-1.18780158  0.33589944]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[-1  1]]
r: [[-1.23449565]]
ev: [[ 0.96221902 -0.27227664]]
lambda: [[0.30299595]]
Scaled r: [[-2.24269983]]
Rounded r: [[-1.]]
Predictions: [[-1.18785519  0.33612432]]
Rounded preds: [[-1.  0.]]
---

Scaled r: [[0.]]
Rounded r: [[0.]]
Predictions: [[0. 0.]]
Rounded preds: [[0. 0.]]
-------
PBT: [[-1  1]]
r: [[-1.23572297]]
ev: [[ 0.96173237 -0.2739906 ]]
lambda: [[0.30287175]]
Scaled r: [[-2.24538977]]
Rounded r: [[-1.]]
Predictions: [[-1.18843478  0.33857648]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[-1  0]]
r: [[-0.96173651]]
ev: [[ 0.96173651 -0.27397605]]
lambda: [[0.30286801]]
Scaled r: [[-1.74754917]]
Rounded r: [[-1.]]
Predictions: [[-0.92493712  0.26349278]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96172503 -0.27401638]]
lambda: [[0.30286344]]
Scaled r: [[0.]]
Rounded r: [[0.]]
Predictions: [[0. 0.]]
Rounded preds: [[0. 0.]]
-------
PBT: [[-1  0]]
r: [[-0.96174491]]
ev: [[ 0.96174491 -0.27394659]]
lambda: [[0.3028597]]
Scaled r: [[-1.74758838]]
Rounded r: [[-1.]]
Predictions: [[-0.92495327  0.26346673]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[0 1]]
r: [[-0.27385972]]
ev: [[ 0.96176965 -0.27385972]]
lambda: [[0.30284445]]
Scaled r: [[-0.49764352]

Scaled r: [[0.]]
Rounded r: [[0.]]
Predictions: [[0. 0.]]
Rounded preds: [[0. 0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96225869 -0.2721364 ]]
lambda: [[0.30258653]]
Scaled r: [[0.]]
Rounded r: [[0.]]
Predictions: [[0. 0.]]
Rounded preds: [[0. 0.]]
-------
PBT: [[0 1]]
r: [[-0.27216502]]
ev: [[ 0.96225059 -0.27216502]]
lambda: [[0.30257145]]
Scaled r: [[-0.49478706]]
Rounded r: [[-0.]]
Predictions: [[-0.26189095  0.0740738 ]]
Rounded preds: [[-0.  0.]]
-------
PBT: [[-1  0]]
r: [[-0.96227225]]
ev: [[ 0.96227225 -0.27208842]]
lambda: [[0.30256775]]
Scaled r: [[-1.74939001]]
Rounded r: [[-1.]]
Predictions: [[-0.92596789  0.26182314]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[0 1]]
r: [[-0.2719968]]
ev: [[ 0.96229816 -0.2719968 ]]
lambda: [[0.30255268]]
Scaled r: [[-0.49449657]]
Rounded r: [[-0.]]
Predictions: [[-0.26174202  0.07398226]]
Rounded preds: [[-0.  0.]]
-------
PBT: [[-1  1]]
r: [[-1.23431996]]
ev: [[ 0.9622883  -0.27203166]]
lambda: [[0.30256854]]
Scaled r: [[-2.2439639]]
R

PBT: [[-1  1]]
r: [[-1.23435519]]
ev: [[ 0.96227442 -0.27208078]]
lambda: [[0.30238287]]
Scaled r: [[-2.2447168]]
Rounded r: [[-1.]]
Predictions: [[-1.18778842  0.33584432]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[-1  0]]
r: [[-0.96226418]]
ev: [[ 0.96226418 -0.27211699]]
lambda: [[0.30237915]]
Scaled r: [[-1.7499208]]
Rounded r: [[-1.]]
Predictions: [[-0.92595234  0.26184843]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96224364 -0.27218961]]
lambda: [[0.30237476]]
Scaled r: [[0.]]
Rounded r: [[0.]]
Predictions: [[0. 0.]]
Rounded preds: [[0. 0.]]
-------
PBT: [[-1  1]]
r: [[-1.23453283]]
ev: [[ 0.96220434 -0.27232848]]
lambda: [[0.30239047]]
Scaled r: [[-2.24501161]]
Rounded r: [[-1.]]
Predictions: [[-1.18787285  0.33619845]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[-1  1]]
r: [[-1.23466263]]
ev: [[ 0.96215308 -0.27250955]]
lambda: [[0.30240619]]
Scaled r: [[-2.24518931]]
Rounded r: [[-1.]]
Predictions: [[-1.18793445  0.33645736]]
Rounded preds: [[-1.  0.]]
-

Rounded preds: [[0. 0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96145207 -0.27497256]]
lambda: [[0.30236213]]
Scaled r: [[0.]]
Rounded r: [[0.]]
Predictions: [[0. 0.]]
Rounded preds: [[0. 0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96142116 -0.27508063]]
lambda: [[0.30235786]]
Scaled r: [[0.]]
Rounded r: [[0.]]
Predictions: [[0. 0.]]
Rounded preds: [[0. 0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96138735 -0.27519878]]
lambda: [[0.30235358]]
Scaled r: [[0.]]
Rounded r: [[0.]]
Predictions: [[0. 0.]]
Rounded preds: [[0. 0.]]
-------
PBT: [[-1  1]]
r: [[-1.23670512]]
ev: [[ 0.9613396  -0.27536552]]
lambda: [[0.30236925]]
Scaled r: [[-2.24904088]]
Rounded r: [[-1.]]
Predictions: [[-1.18889361  0.34054595]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[-1  1]]
r: [[-1.23684615]]
ev: [[ 0.96128296 -0.27556319]]
lambda: [[0.30238492]]
Scaled r: [[-2.24923906]]
Rounded r: [[-1.]]
Predictions: [[-1.18895913  0.34082928]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96123272

PBT: [[-1  0]]
r: [[-0.96045966]]
ev: [[ 0.96045966 -0.27841918]]
lambda: [[0.30220717]]
Scaled r: [[-1.74713613]]
Rounded r: [[-1.]]
Predictions: [[-0.92248276  0.26741039]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96045269 -0.27844324]]
lambda: [[0.30220304]]
Scaled r: [[0.]]
Rounded r: [[0.]]
Predictions: [[0. 0.]]
Rounded preds: [[0. 0.]]
-------
PBT: [[0 1]]
r: [[-0.27841682]]
ev: [[ 0.96046034 -0.27841682]]
lambda: [[0.3021882]]
Scaled r: [[-0.50647349]]
Rounded r: [[-0.]]
Predictions: [[-0.26740832  0.07751593]]
Rounded preds: [[-0.  0.]]
-------
PBT: [[-1  0]]
r: [[-0.96049207]]
ev: [[ 0.96049207 -0.27830735]]
lambda: [[0.30218424]]
Scaled r: [[-1.74726138]]
Rounded r: [[-1.]]
Predictions: [[-0.92254502  0.267312  ]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[0 1]]
r: [[-0.27819565]]
ev: [[ 0.96052443 -0.27819565]]
lambda: [[0.30216942]]
Scaled r: [[-0.50608687]]
Rounded r: [[-0.]]
Predictions: [[-0.26721371  0.07739282]]
Rounded preds: [[-0.  0.]]
----

PBT: [[-1  1]]
r: [[-1.23852212]]
ev: [[ 0.96060508 -0.27791703]]
lambda: [[0.30203833]]
Scaled r: [[-2.25357874]]
Rounded r: [[-1.]]
Predictions: [[-1.18973064  0.34420639]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[-1  0]]
r: [[-0.96059788]]
ev: [[ 0.96059788 -0.27794191]]
lambda: [[0.30203437]]
Scaled r: [[-1.74788735]]
Rounded r: [[-1.]]
Predictions: [[-0.92274829  0.26699041]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96057917 -0.27800656]]
lambda: [[0.30203029]]
Scaled r: [[0.]]
Rounded r: [[0.]]
Predictions: [[0. 0.]]
Rounded preds: [[0. 0.]]
-------
PBT: [[-1  1]]
r: [[-1.23867888]]
ev: [[ 0.96054122 -0.27813766]]
lambda: [[0.30204576]]
Scaled r: [[-2.25383626]]
Rounded r: [[-1.]]
Predictions: [[-1.18980213  0.34452324]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[0 1]]
r: [[-0.27817966]]
ev: [[ 0.96052906 -0.27817966]]
lambda: [[0.30203103]]
Scaled r: [[-0.50617372]]
Rounded r: [[-0.]]
Predictions: [[-0.26719965  0.07738393]]
Rounded preds: [[-0.  0.]]
-

r: [[-0.96128898]]
ev: [[ 0.96128898 -0.2755422 ]]
lambda: [[0.3015987]]
Scaled r: [[-1.75040775]]
Rounded r: [[-1.]]
Predictions: [[-0.9240765   0.26487568]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[-1  0]]
r: [[-0.961344]]
ev: [[ 0.961344   -0.27535016]]
lambda: [[0.30159485]]
Scaled r: [[-1.75051911]]
Rounded r: [[-1.]]
Predictions: [[-0.92418229  0.26470622]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[-1  0]]
r: [[-0.96140565]]
ev: [[ 0.96140565 -0.27513485]]
lambda: [[0.30159101]]
Scaled r: [[-1.75064251]]
Rounded r: [[-1.]]
Predictions: [[-0.92430082  0.26451619]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[-1  1]]
r: [[-1.23650582]]
ev: [[ 0.96141954 -0.27508628]]
lambda: [[0.30160626]]
Scaled r: [[-2.25152093]]
Rounded r: [[-1.]]
Predictions: [[-1.18880086  0.34014579]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96141448 -0.27510397]]
lambda: [[0.30160222]]
Scaled r: [[0.]]
Rounded r: [[0.]]
Predictions: [[0. 0.]]
Rounded preds: [[0. 0.]]
-------
PBT: [[0 

r: [[0.]]
ev: [[ 0.96209442 -0.27271655]]
lambda: [[0.30140063]]
Scaled r: [[0.]]
Rounded r: [[0.]]
Predictions: [[0. 0.]]
Rounded preds: [[0. 0.]]
-------
PBT: [[0 1]]
r: [[-0.2726373]]
ev: [[ 0.96211689 -0.2726373 ]]
lambda: [[0.30138625]]
Scaled r: [[-0.49661925]]
Rounded r: [[-0.]]
Predictions: [[-0.26230895  0.0743311 ]]
Rounded preds: [[-0.  0.]]
-------
PBT: [[-1  0]]
r: [[-0.96215723]]
ev: [[ 0.96215723 -0.27249487]]
lambda: [[0.30138245]]
Scaled r: [[-1.7526172]]
Rounded r: [[-1.]]
Predictions: [[-0.92574654  0.26218291]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[0 1]]
r: [[-0.27236054]]
ev: [[ 0.96219527 -0.27236054]]
lambda: [[0.30136809]]
Scaled r: [[-0.49613007]]
Rounded r: [[-0.]]
Predictions: [[-0.26206402  0.07418026]]
Rounded preds: [[-0.  0.]]
-------
PBT: [[-1  0]]
r: [[-0.96224562]]
ev: [[ 0.96224562 -0.27218258]]
lambda: [[0.3013643]]
Scaled r: [[-1.75283099]]
Rounded r: [[-1.]]
Predictions: [[-0.92591664  0.2619065 ]]
Rounded preds: [[-1.  0.]]
-------
PBT: [[-1  

In [202]:
def non_linearity(array):
    """Apply a non-linearity to array."""
    signs = np.sign(array)
    rand_vals = np.random.uniform(size=array.shape)
    binary_values = np.where(np.abs(array) > rand_vals, 1, 0)
    return binary_values*signs

In [203]:
# Loop
sum_preds = 0
sum_inputs = 0
for _ in range(0, 1000):
    ternary_input = signal_pre_processor(data_in, mean)
    sum_inputs += ternary_input
    vpu.update_cov(ternary_input)
    vpu.pi.iterate(cov=vpu.cu.covariance)
    r = vpu.forward(ternary_input)
    pred = vpu.backward(r) 
    sum_preds += pred
    # Check values
    print(
        f"PBT: {ternary_input.T}", 
        f"r: {r}", 
        f"ev: {vpu.pi.eigenvector.T}",
        f"lambda: {vpu.pi.eigenvalue}",
        f"Scaled r: {r*np.sqrt(vpu.pi.eigenvalue)}", 
        f"NL r: {non_linearity(r)}", 
        f"Predictions: {pred.T}", 
        f"NL preds: {non_linearity(pred).T}", 
        sep="\n", end="\n-------\n")

PBT: [[-1  0]]
r: [[-0.96213033]]
ev: [[ 0.96213033 -0.27258985]]
lambda: [[0.30138808]]
Scaled r: [[-0.52819823]]
NL r: [[-1.]]
Predictions: [[-0.92569477  0.26226696]]
NL preds: [[-1.  0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96211145 -0.27265648]]
lambda: [[0.30138412]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96208541 -0.27274835]]
lambda: [[0.30138016]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
-------
PBT: [[-1  0]]
r: [[-0.9620945]]
ev: [[ 0.9620945  -0.27271628]]
lambda: [[0.30137635]]
Scaled r: [[-0.52816828]]
NL r: [[-1.]]
Predictions: [[-0.92562583  0.26237883]]
NL preds: [[-1.  0.]]
-------
PBT: [[-1  1]]
r: [[-1.23485997]]
ev: [[ 0.96207504 -0.27278493]]
lambda: [[0.30139127]]
Scaled r: [[-0.67792719]]
NL r: [[-1.]]
Predictions: [[-1.18802796  0.3368512 ]]
NL preds: [[-1.  0.]]
-------
PBT: [[-1  1]]
r: [[-1.23495562]]
ev: [[ 0.96203717 -0.27291845]]
lambda: [[0.30140

PBT: [[-1  1]]
r: [[-1.23412752]]
ev: [[ 0.96236409 -0.27176343]]
lambda: [[0.3011693]]
Scaled r: [[-0.67727554]]
NL r: [[-1.]]
Predictions: [[-1.18768001  0.33539073]]
NL preds: [[-1.  1.]]
-------
PBT: [[0 1]]
r: [[-0.27178913]]
ev: [[ 0.96235683 -0.27178913]]
lambda: [[0.30115508]]
Scaled r: [[-0.14915135]]
NL r: [[-0.]]
Predictions: [[-0.26155813  0.07386933]]
NL preds: [[-0.  0.]]
-------
PBT: [[-1  1]]
r: [[-1.23422161]]
ev: [[ 0.96232705 -0.27189457]]
lambda: [[0.30116982]]
Scaled r: [[-0.67732775]]
NL r: [[-1.]]
Predictions: [[-1.18772484  0.33557815]]
NL preds: [[-1.  0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96229399 -0.27201154]]
lambda: [[0.30116594]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96225882 -0.27213594]]
lambda: [[0.30116206]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
-------
PBT: [[-1  1]]
r: [[-1.23451601]]
ev: [[ 0.96221098 -0.27230502]]
lambda: [[0.3011768

PBT: [[-1  1]]
r: [[-1.23546809]]
ev: [[ 0.96183381 -0.27363428]]
lambda: [[0.30097564]]
Scaled r: [[-0.6777932]]
NL r: [[-1.]]
Predictions: [[-1.18831498  0.33806642]]
NL preds: [[-1.  0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96181319 -0.27370675]]
lambda: [[0.30097186]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
-------
PBT: [[-1  0]]
r: [[-0.96182548]]
ev: [[ 0.96182548 -0.27366355]]
lambda: [[0.30096799]]
Scaled r: [[-0.52766275]]
NL r: [[-1.]]
Predictions: [[-0.92510826  0.26321658]]
NL preds: [[-1.  0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96181959 -0.27368427]]
lambda: [[0.30096421]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
-------
PBT: [[-1  1]]
r: [[-1.23557633]]
ev: [[ 0.96179076 -0.27378557]]
lambda: [[0.30097887]]
Scaled r: [[-0.67785621]]
NL r: [[-1.]]
Predictions: [[-1.18836589  0.33828297]]
NL preds: [[-1.  1.]]
-------
PBT: [[0 1]]
r: [[-0.27380884]]
ev: [[ 0.96178413 -0.27380884]]
lambda: [[0.3009647

PBT: [[0 1]]
r: [[-0.27264081]]
ev: [[ 0.96211589 -0.27264081]]
lambda: [[0.30085559]]
Scaled r: [[-0.14954432]]
NL r: [[-0.]]
Predictions: [[-0.26231206  0.07433301]]
NL preds: [[-0.  0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96210641 -0.27267426]]
lambda: [[0.30085186]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
-------
PBT: [[-1  0]]
r: [[-0.96212568]]
ev: [[ 0.96212568 -0.27260628]]
lambda: [[0.300848]]
Scaled r: [[-0.52772221]]
NL r: [[-1.]]
Predictions: [[-0.92568582  0.2622815 ]]
NL preds: [[-1.  1.]]
-------
PBT: [[-1  1]]
r: [[-1.23476314]]
ev: [[ 0.96211335 -0.27264979]]
lambda: [[0.30086248]]
Scaled r: [[-0.6772791]]
NL r: [[-1.]]
Predictions: [[-1.1879821   0.33665791]]
NL preds: [[-1.  0.]]
-------
PBT: [[-1  1]]
r: [[-1.23484607]]
ev: [[ 0.96208054 -0.27276553]]
lambda: [[0.30087696]]
Scaled r: [[-0.67734089]]
NL r: [[-1.]]
Predictions: [[-1.18802137  0.33682344]]
NL preds: [[-1.  1.]]
-------
PBT: [[0 1]]
r: [[-0.2727981]]
ev: [[ 0.962071

PBT: [[-1  1]]
r: [[-1.23480903]]
ev: [[ 0.96209519 -0.27271384]]
lambda: [[0.30066463]]
Scaled r: [[-0.67708153]]
NL r: [[-1.]]
Predictions: [[-1.18800384  0.33674951]]
NL preds: [[-1.  0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96205041 -0.27287177]]
lambda: [[0.30066098]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96200755 -0.27302285]]
lambda: [[0.30065734]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
-------
PBT: [[0 1]]
r: [[-0.27307862]]
ev: [[ 0.96199172 -0.27307862]]
lambda: [[0.30064341]]
Scaled r: [[-0.14973163]]
NL r: [[-0.]]
Predictions: [[-0.26269937  0.07457193]]
NL preds: [[-1.  0.]]
-------
PBT: [[0 1]]
r: [[-0.27307254]]
ev: [[ 0.96199345 -0.27307254]]
lambda: [[0.30062949]]
Scaled r: [[-0.14972482]]
NL r: [[-0.]]
Predictions: [[-0.26269399  0.07456861]]
NL preds: [[-0.  0.]]
-------
PBT: [[0 1]]
r: [[-0.27302629]]
ev: [[ 0.96200657 -0.27302629]]
lambda: [[0.30061556]]

PBT: [[0 1]]
r: [[-0.27619629]]
ev: [[ 0.96110125 -0.27619629]]
lambda: [[0.30060992]]
Scaled r: [[-0.15143264]]
NL r: [[-0.]]
Predictions: [[-0.2654526   0.07628439]]
NL preds: [[-0.  0.]]
-------
PBT: [[-1  1]]
r: [[-1.23735209]]
ev: [[ 0.96107924 -0.27627285]]
lambda: [[0.30062427]]
Scaled r: [[-0.67843043]]
NL r: [[-1.]]
Predictions: [[-1.18919341  0.34184679]]
NL preds: [[-1.  0.]]
-------
PBT: [[-1  1]]
r: [[-1.23744867]]
ev: [[ 0.96104027 -0.27640841]]
lambda: [[0.30063862]]
Scaled r: [[-0.67849957]]
NL r: [[-1.]]
Predictions: [[-1.189238    0.34204122]]
NL preds: [[-1.  0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96100108 -0.27654462]]
lambda: [[0.30063509]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
-------
PBT: [[-1  0]]
r: [[-0.96100108]]
ev: [[ 0.96100108 -0.27654463]]
lambda: [[0.30063108]]
Scaled r: [[-0.52691531]]
NL r: [[-1.]]
Predictions: [[-0.92352307  0.26575968]]
NL preds: [[-1.  0.]]
-------
PBT: [[-1  1]]
r: [[-1.23760682]]
ev: [[ 0.

PBT: [[-1  0]]
r: [[-0.96153591]]
ev: [[ 0.96153591 -0.27467924]]
lambda: [[0.30046199]]
Scaled r: [[-0.52706027]]
NL r: [[-1.]]
Predictions: [[-0.92455131  0.26411396]]
NL preds: [[-1.  0.]]
-------
PBT: [[-1  1]]
r: [[-1.23623519]]
ev: [[ 0.9615279  -0.27470729]]
lambda: [[0.3004761]]
Scaled r: [[-0.67765098]]
NL r: [[-1.]]
Predictions: [[-1.18867463  0.33960282]]
NL preds: [[-1.  1.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96150887 -0.2747739 ]]
lambda: [[0.30047264]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
-------
PBT: [[-1  1]]
r: [[-1.2363747]]
ev: [[ 0.96147207 -0.27490262]]
lambda: [[0.30048675]]
Scaled r: [[-0.67773946]]
NL r: [[-1.]]
Predictions: [[-1.18873974  0.33988265]]
NL preds: [[-1.  0.]]
-------
PBT: [[-1  0]]
r: [[-0.96147326]]
ev: [[ 0.96147326 -0.27489848]]
lambda: [[0.30048275]]
Scaled r: [[-0.52704413]]
NL r: [[-1.]]
Predictions: [[-0.92443083  0.26430754]]
NL preds: [[-1.  1.]]
-------
PBT: [[0 1]]
r: [[-0.27485375]]
ev: [[ 0.96

PBT: [[-1  1]]
r: [[-1.23665493]]
ev: [[ 0.96135974 -0.27529519]]
lambda: [[0.30039488]]
Scaled r: [[-0.67778944]]
NL r: [[-1.]]
Predictions: [[-1.18887027  0.34044515]]
NL preds: [[-1.  1.]]
-------
PBT: [[0 1]]
r: [[-0.27533428]]
ev: [[ 0.96134855 -0.27533428]]
lambda: [[0.30038119]]
Scaled r: [[-0.15090257]]
NL r: [[-0.]]
Predictions: [[-0.26469221  0.07580896]]
NL preds: [[-0.  0.]]
-------
PBT: [[0 1]]
r: [[-0.2753178]]
ev: [[ 0.96135327 -0.2753178 ]]
lambda: [[0.3003675]]
Scaled r: [[-0.1508901]]
NL r: [[-1.]]
Predictions: [[-0.26467766  0.07579989]]
NL preds: [[-0.  0.]]
-------
PBT: [[0 1]]
r: [[-0.27526512]]
ev: [[ 0.96136836 -0.27526512]]
lambda: [[0.30035382]]
Scaled r: [[-0.1508578]]
NL r: [[-0.]]
Predictions: [[-0.26463117  0.07577089]]
NL preds: [[-0.  0.]]
-------
PBT: [[0 1]]
r: [[-0.27518884]]
ev: [[ 0.96139019 -0.27518884]]
lambda: [[0.30034013]]
Scaled r: [[-0.15081256]]
NL r: [[-0.]]
Predictions: [[-0.26456386  0.0757289 ]]
NL preds: [[-0.  0.]]
-------
PBT: [[-1  1

NL r: [[-1.]]
Predictions: [[-1.19032176  0.3468469 ]]
NL preds: [[-1.  0.]]
-------
PBT: [[0 1]]
r: [[-0.27972193]]
ev: [[ 0.96008106 -0.27972193]]
lambda: [[0.30026132]]
Scaled r: [[-0.15327673]]
NL r: [[-0.]]
Predictions: [[-0.26855573  0.07824436]]
NL preds: [[-0.  0.]]
-------
PBT: [[-1  0]]
r: [[-0.96011249]]
ev: [[ 0.96011249 -0.27961403]]
lambda: [[0.30025715]]
Scaled r: [[-0.5261006]]
NL r: [[-1.]]
Predictions: [[-0.92181599  0.26846093]]
NL preds: [[-1.  1.]]
-------
PBT: [[-1  1]]
r: [[-1.23973578]]
ev: [[ 0.96010868 -0.2796271 ]]
lambda: [[0.30027115]]
Scaled r: [[-0.67933805]]
NL r: [[-1.]]
Predictions: [[-1.19028109  0.34666372]]
NL preds: [[-1.  1.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.96009224 -0.27968356]]
lambda: [[0.3002679]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
-------
PBT: [[0 1]]
r: [[-0.27967911]]
ev: [[ 0.96009353 -0.27967911]]
lambda: [[0.30025426]]
Scaled r: [[-0.15325146]]
NL r: [[-0.]]
Predictions: [[-0.26851811  0.078

PBT: [[-1  1]]
r: [[-1.24165856]]
ev: [[ 0.95931267 -0.28234588]]
lambda: [[0.30023187]]
Scaled r: [[-0.68034717]]
NL r: [[-1.]]
Predictions: [[-1.19113879  0.35057718]]
NL preds: [[-1.  0.]]
-------
PBT: [[0 1]]
r: [[-0.2823474]]
ev: [[ 0.95931223 -0.2823474 ]]
lambda: [[0.30021831]]
Scaled r: [[-0.1547043]]
NL r: [[-1.]]
Predictions: [[-0.27085931  0.07972005]]
NL preds: [[-1.  0.]]
-------
PBT: [[-1  1]]
r: [[-1.24171839]]
ev: [[ 0.95928771 -0.28243067]]
lambda: [[0.30023221]]
Scaled r: [[-0.68038034]]
NL r: [[-1.]]
Predictions: [[-1.19116519  0.35069936]]
NL preds: [[-1.  0.]]
-------
PBT: [[0 1]]
r: [[-0.28244409]]
ev: [[ 0.95928376 -0.28244409]]
lambda: [[0.30021865]]
Scaled r: [[-0.15475737]]
NL r: [[-0.]]
Predictions: [[-0.27094403  0.07977467]]
NL preds: [[-0.  0.]]
-------
PBT: [[-1  0]]
r: [[-0.95930642]]
ev: [[ 0.95930642 -0.28236713]]
lambda: [[0.30021437]]
Scaled r: [[-0.52562146]]
NL r: [[-1.]]
Predictions: [[-0.92026881  0.2708766 ]]
NL preds: [[-1.  0.]]
-------
PBT: [

Is it here just learing to replicate the top entry?

No - our prediction is 0.2 - meaning we have a 1 20% of the time.
* Do we need to scale the R value using the eigenvalue?
* Do we need to scale the predicted outputs using the eigenvalue?
* Do we need to apply the probabilistic computation on the output?

Rounding is not useful - ignore - we need probabilities. Predictions here appear like probabilities - 0.23 means that 1 occurs 23% of the time - -0.23 indicates -1 occurs with 23%.

Some of the predictions are > 1 or -1 - e.g. -1.18307985. r is also sometimes > 1 or -1, e.g. -1.22. This happens when we have [-1, 1]

In [208]:
input_estimate = (sum_inputs / (1000 / mean))+mean
pred_estimate = (sum_preds / (1000 / mean))+mean
residual = input_estimate - pred_estimate
print(f"Input estimate = {input_estimate.T};\nPredicted estimate = {pred_estimate.T};\nresidual = {residual.T}\n")

Input estimate = [[ 61.824 198.144]];
Predicted estimate = [[ 48.26758964 150.87170251]];
residual = [[13.55641036 47.27229749]]



This makes sense - the VPU was better at detecting the first entry than the second.

In [336]:
def add_to_array(array, frame):
    """Add a frame to a rolling array."""
    array = np.roll(array, -1, axis=1)
    # Add frame to end of buffer
    array[..., -1] = frame.flatten()
    return array

class test_vpu_wrapper:
    """Test wrapper for VPU.
    
    Proto - stage/stack. We store 1000 samples in the buffer.
    """
    def __init__(self, vpu, buf_length=1000):
        self.vpu = vpu
        # Temp fields for output
        self.input_buffer = np.zeros(shape=(vpu.size, buf_length))
        self.pred_buffer = np.zeros(shape=(vpu.size, buf_length))
        self.nl_pred_buffer = np.zeros(shape=(vpu.size, buf_length))
        self.r_buffer = np.zeros(shape=(1, buf_length))
        self.residual_buffer = np.zeros(shape=(vpu.size, buf_length))
    
    def iterate(self, input_signal):
        # Update covariance data of VPU
        self.vpu.update_cov(input_signal)
        # Get current covariance matrix
        cov = self.vpu.cu.covariance
        # Power iterate using covariance matrix
        self.vpu.pi.iterate(cov=cov)
        # Get r
        r = self.vpu.forward(input_signal)
        pred = self.vpu.backward(r)
        pred_nl = non_linearity(pred).astype(np.int8)
        
        # Calculate residual
        residual = input_signal.astype(np.int8) - pred_nl
        
        # Store last input and pred for output repr
        # Create rolling sum of inputs
        self.input_buffer = add_to_array(self.input_buffer, input_signal)
        # Add to rolling sum of predictions
        self.pred_buffer = add_to_array(self.pred_buffer, pred)
        self.nl_pred_buffer = add_to_array(self.nl_pred_buffer, pred_nl)
        self.r_buffer = add_to_array(self.r_buffer, r)
        self.residual_buffer =  add_to_array(self.residual_buffer, residual)
        return r, pred, residual
    
    @property
    def input_estimate(self):
        # How do we get the mean in here?
        return (self.input_buffer.sum(axis=1).T / 1000).reshape(-1, 1)
    
    @property
    def pred_estimate(self):
        # How do we get the mean in here?
        return (self.pred_buffer.sum(axis=1).T / 1000).reshape(-1, 1)
    
    @property
    def error(self):
        return self.input_estimate - self.pred_estimate
    
    def __repr__(self):
        return f"""
                Input: {self.input_buffer[:,-1].T}
                r: {self.r_buffer[:,-1]}
                ev: {self.vpu.pi.eigenvector.T}
                lambda: {self.vpu.pi.eigenvalue}
                Scaled r: {self.r_buffer[:,-1]*np.sqrt(self.vpu.pi.eigenvalue)}
                NL r: {non_linearity(self.r_buffer[:,-1])}
                Predictions: {self.pred_buffer[:,-1].T}
                NL Predictions: {self.nl_pred_buffer[:,-1].T}
                Residual: {self.residual_buffer[:,-1].T}
                ------------------------------
        """

In [337]:
wrapper_1 = test_vpu_wrapper(vpu_1)
wrapper_2 = test_vpu_wrapper(vpu_2)

In [358]:
for _ in range(0, 1000):
    # First VPU
    ternary_input = signal_pre_processor(data_in, mean)
    r, pred, residual = wrapper_1.iterate(ternary_input)
    # print(residual)
    # print(ternary_input, pred, ternary_input-pred)
    print(f"Wrapper - 1 {wrapper_1}")
    r, pred, residual = wrapper_2.iterate(residual)
    print(f"Wrapper - 2 {wrapper_2}")

Wrapper - 1 
                Input: [0. 1.]
                r: [0.67103171]
                ev: [[-0.74142865  0.67103171]]
                lambda: [[0.25094915]]
                Scaled r: [[0.33615216]]
                NL r: [0.]
                Predictions: [-0.49752214  0.45028355]
                NL Predictions: [0. 0.]
                Residual: [0. 1.]
                ------------------------------
        
Wrapper - 2 
                Input: [0. 1.]
                r: [0.71116841]
                ev: [[0.70302169 0.71116841]]
                lambda: [[25.49416904]]
                Scaled r: [[3.59081382]]
                NL r: [0.]
                Predictions: [0.49996682 0.50576051]
                NL Predictions: [1. 1.]
                Residual: [-1.  0.]
                ------------------------------
        
Wrapper - 1 
                Input: [0. 1.]
                r: [0.67098981]
                ev: [[-0.74146657  0.67098981]]
                lambda: [[0.25094007]]
      

Wrapper - 2 
                Input: [0. 0.]
                r: [0.]
                ev: [[0.70297745 0.71121214]]
                lambda: [[25.44370715]]
                Scaled r: [[0.]]
                NL r: [0.]
                Predictions: [0. 0.]
                NL Predictions: [0. 0.]
                Residual: [0. 0.]
                ------------------------------
        
Wrapper - 1 
                Input: [0. 1.]
                r: [0.66978968]
                ev: [[-0.74255086  0.66978968]]
                lambda: [[0.25092628]]
                Scaled r: [[0.33551468]]
                NL r: [1.]
                Predictions: [-0.49735291  0.44861822]
                NL Predictions: [-1.  1.]
                Residual: [1. 0.]
                ------------------------------
        
Wrapper - 2 
                Input: [1. 0.]
                r: [0.70297616]
                ev: [[0.70297616 0.71121341]]
                lambda: [[25.4428071]]
                Scaled r: [[3.54587245]]

Wrapper - 1 
                Input: [-1.  0.]
                r: [0.74179742]
                ev: [[-0.74179742  0.67062403]]
                lambda: [[0.25090493]]
                Scaled r: [[0.37156938]]
                NL r: [1.]
                Predictions: [-0.55026342  0.49746717]
                NL Predictions: [-1.  0.]
                Residual: [0. 0.]
                ------------------------------
        
Wrapper - 2 
                Input: [0. 0.]
                r: [0.]
                ev: [[0.70292095 0.71126798]]
                lambda: [[25.39707272]]
                Scaled r: [[0.]]
                NL r: [0.]
                Predictions: [0. 0.]
                NL Predictions: [0. 0.]
                Residual: [0. 0.]
                ------------------------------
        
Wrapper - 1 
                Input: [-1.  1.]
                r: [1.41242579]
                ev: [[-0.7417565   0.67066929]]
                lambda: [[0.250912]]
                Scaled r: [[0.707499

Wrapper - 1 
                Input: [-1.  0.]
                r: [0.74009773]
                ev: [[-0.74009773  0.67249933]]
                lambda: [[0.25099092]]
                Scaled r: [[0.37078152]]
                NL r: [1.]
                Predictions: [-0.54774465  0.49771523]
                NL Predictions: [0. 0.]
                Residual: [-1.  0.]
                ------------------------------
        
Wrapper - 2 
                Input: [-1.  0.]
                r: [-0.70287271]
                ev: [[0.70287271 0.71131565]]
                lambda: [[25.34401914]]
                Scaled r: [[-3.53846112]]
                NL r: [-0.]
                Predictions: [-0.49403005 -0.49996436]
                NL Predictions: [-1. -1.]
                Residual: [0. 1.]
                ------------------------------
        
Wrapper - 1 
                Input: [0. 1.]
                r: [0.67249112]
                ev: [[-0.74010519  0.67249112]]
                lambda: [[0.250981

Wrapper - 2 
                Input: [-1.  0.]
                r: [-0.70282805]
                ev: [[0.70282805 0.71135977]]
                lambda: [[25.29264108]]
                Scaled r: [[-3.53464806]]
                NL r: [-0.]
                Predictions: [-0.49396727 -0.4999636 ]
                NL Predictions: [ 0. -1.]
                Residual: [-1.  1.]
                ------------------------------
        
Wrapper - 1 
                Input: [-1.  0.]
                r: [0.74032554]
                ev: [[-0.74032554  0.67224854]]
                lambda: [[0.25104524]]
                Scaled r: [[0.37093578]]
                NL r: [0.]
                Predictions: [-0.5480819   0.49768276]
                NL Predictions: [0. 1.]
                Residual: [-1. -1.]
                ------------------------------
        
Wrapper - 2 
                Input: [-1. -1.]
                r: [-1.41418781]
                ev: [[0.70282661 0.7113612 ]]
                lambda: [[25.29

...

Wrapper - 2 
                Input: [-1.  0.]
                r: [-0.70244412]
                ev: [[0.70244412 0.7117389 ]]
                lambda: [[24.8987999]]
                Scaled r: [[-3.50510464]]
                NL r: [-1.]
                Predictions: [-0.49342774 -0.4999568 ]
                NL Predictions: [-1. -1.]
                Residual: [0. 1.]
                ------------------------------
        
Wrapper - 1 
                Input: [-1.  1.]
                r: [1.41293188]
                ev: [[-0.73656366  0.67636822]]
                lambda: [[0.25124706]]
                Scaled r: [[0.70822576]]
                NL r: [1.]
                Predictions: [-1.04071429  0.95566222]
                NL Predictions: [-1.  1.]
                Residual: [0. 0.]
                ------------------------------
        
Wrapper - 2 
                Input: [0. 0.]
                r: [0.]
                ev: [[0.70244334 0.71173967]]
                lambda: [[24.89800694]]
     

        
Wrapper - 1 
                Input: [-1.  0.]
                r: [0.73680169]
                ev: [[-0.73680169  0.67610891]]
                lambda: [[0.25118904]]
                Scaled r: [[0.3692759]]
                NL r: [1.]
                Predictions: [-0.54287674  0.49815819]
                NL Predictions: [-1.  1.]
                Residual: [ 0. -1.]
                ------------------------------
        
Wrapper - 2 
                Input: [ 0. -1.]
                r: [-0.71177988]
                ev: [[0.70240259 0.71177988]]
                lambda: [[24.85559854]]
                Scaled r: [[-3.54860632]]
                NL r: [-1.]
                Predictions: [-0.49995603 -0.5066306 ]
                NL Predictions: [ 0. -1.]
                Residual: [0. 0.]
                ------------------------------
        
Wrapper - 1 
                Input: [-1.  0.]
                r: [0.73679711]
                ev: [[-0.73679711  0.67611391]]
                lambda

Wrapper - 1 
                Input: [0. 0.]
                r: [0.]
                ev: [[-0.73780934  0.67500917]]
                lambda: [[0.25114849]]
                Scaled r: [[0.]]
                NL r: [0.]
                Predictions: [0. 0.]
                NL Predictions: [0. 0.]
                Residual: [0. 0.]
                ------------------------------
        
Wrapper - 2 
                Input: [0. 0.]
                r: [0.]
                ev: [[0.70235368 0.71182815]]
                lambda: [[24.80932326]]
                Scaled r: [[0.]]
                NL r: [0.]
                Predictions: [0. 0.]
                NL Predictions: [0. 0.]
                Residual: [0. 0.]
                ------------------------------
        
Wrapper - 1 
                Input: [-1.  0.]
                r: [0.73781467]
                ev: [[-0.73781467  0.67500335]]
                lambda: [[0.25113948]]
                Scaled r: [[0.3697471]]
                NL r: [0.]
     

Wrapper - 1 
                Input: [-1.  0.]
                r: [0.73935976]
                ev: [[-0.73935976  0.67331058]]
                lambda: [[0.25102784]]
                Scaled r: [[0.37043904]]
                NL r: [1.]
                Predictions: [-0.54665286  0.49781875]
                NL Predictions: [0. 1.]
                Residual: [-1. -1.]
                ------------------------------
        
Wrapper - 2 
                Input: [-1. -1.]
                r: [-1.41418128]
                ev: [[0.70231276 0.71186851]]
                lambda: [[24.76372962]]
                Scaled r: [[-7.03741416]]
                NL r: [-1.]
                Predictions: [-0.99319756 -1.00671112]
                NL Predictions: [-1. -1.]
                Residual: [0. 0.]
                ------------------------------
        
Wrapper - 1 
                Input: [0. 0.]
                r: [0.]
                ev: [[-0.73938765  0.67327996]]
                lambda: [[0.25103885]]
   

Wrapper - 1 
                Input: [-1.  1.]
                r: [1.41251796]
                ev: [[-0.74087478  0.67164317]]
                lambda: [[0.25095792]]
                Scaled r: [[0.70761077]]
                NL r: [1.]
                Predictions: [-1.04649894  0.94870805]
                NL Predictions: [-1.  1.]
                Residual: [0. 0.]
                ------------------------------
        
Wrapper - 2 
                Input: [0. 0.]
                r: [0.]
                ev: [[0.70227779 0.71190302]]
                lambda: [[24.71662397]]
                Scaled r: [[0.]]
                NL r: [0.]
                Predictions: [0. 0.]
                NL Predictions: [0. 0.]
                Residual: [0. 0.]
                ------------------------------
        
Wrapper - 1 
                Input: [-1.  1.]
                r: [1.41251234]
                ev: [[-0.74092929  0.67158305]]
                lambda: [[0.25096476]]
                Scaled r: [[0.7076

There's an error in the residual - [-1, 0] - [-1, 1] is being = [0, 0]. Data type? No was because we were storing non binary last pred.

Here the input of wrapper 2 is not syncing up with the residual from wrapper 1. Wrapper 2 input is wrapper 1 input. Typo in print statement.

This needs a lot of testing as it is easy for errors to creep in.

AND YES we can get 2 and -2 as residual input, which causes the eigenvalue to explode. **Do we need to rescale the residuals to between -1 and 1? How would we do this? Clamping? Or scaling? Or scaling then probability - e.g. a difference of 1 > 0.5 > 50% of 1.

r and ev seem quite unstable > flipping sign.

In [359]:
(wrapper_1.input_buffer.sum(axis=1) / 1000).shape

(2,)

In [360]:
data_in

array([[ 61],
       [196]])

In [361]:
print(wrapper_1.input_estimate, wrapper_1.pred_estimate)

[[-0.532]
 [ 0.543]] [[-0.56088619]
 [ 0.51137302]]


In [362]:
print(wrapper_1.input_estimate*mean + mean, wrapper_1.pred_estimate*mean + mean)

[[ 59.904]
 [197.504]] [[ 56.20656717]
 [193.45574666]]


In [363]:
print(wrapper_1.error, wrapper_1.error*mean)

[[0.02888619]
 [0.03162698]] [[3.69743283]
 [4.04825334]]


In [364]:
print(wrapper_2.input_estimate, wrapper_2.pred_estimate)

[[0.024]
 [0.044]] [[0.03383753]
 [0.03428162]]


In [365]:
print(wrapper_2.input_estimate*mean, wrapper_2.pred_estimate*mean)

[[3.072]
 [5.632]] [[4.3312038 ]
 [4.38804735]]


Wrapper 2 is fairly out. Are we calculating properly here?

Is this because we are losing information when binarising r?

One run is closer - wrapper 2 estimate os [4.5, 10.2] and error is [7.7, 10]

In [366]:
(wrapper_1.pred_estimate*mean+mean)+(wrapper_2.pred_estimate*mean)

array([[ 60.53777097],
       [197.84379401]])

The above:
```array([[ 58.60746796],
       [194.18347364]])
```
is close.

In [367]:
print(wrapper_2.error, wrapper_2.error*mean)

[[-0.00983753]
 [ 0.00971838]] [[-1.2592038 ]
 [ 1.24395265]]


**The wrapper 2 error is much lower than the wrapper 1 error. But when summing the signals we don't get the original signal. Why is this?**

**Do we need to convert the wrapper_1 error into the original input space?**

In [256]:
print(wrapper_1.count, wrapper_1.sum_inputs)

1000 [[-518]
 [ 563]]


In [257]:
 mean = np.asarray([128,128]).reshape(-1, 1); print(mean)

[[128]
 [128]]


In [258]:
wrapper_1.sum_inputs / (wrapper_1.count / mean)

array([[-66.304],
       [ 72.064]])

In [259]:
(wrapper_1.sum_inputs / wrapper_1.count)*mean

array([[-66.304],
       [ 72.064]])

In [226]:
ternary_input.dtype

dtype('int64')

In [223]:
pred.dtype; print(pred)

[[0.]
 [0.]]


In [224]:
residual.dtype; print(residual)

[[0.]
 [0.]]


In [219]:
np.asarray([[0,  1]])-np.asarray([[-0.,  1.]])

array([[0., 0.]])

In [209]:
# Test with two VPUs
# Loop
sum_preds = 0
sum_inputs = 0
for _ in range(0, 1000):
    # First VPU
    ternary_input = signal_pre_processor(data_in, mean)
    sum_inputs += ternary_input
    vpu_1.update_cov(ternary_input)
    vpu_1.pi.iterate(cov=vpu_1.cu.covariance)
    r = vpu_1.forward(ternary_input)
    pred = vpu_1.backward(r) 
    sum_preds += pred
    residual = ternary_input - non_linearity(pred) # Might this be outside range - -1, 0, 1?
    # Second VPU
    vpu_2.update_cov(residual)
    vpu_2.pi.iterate(cov=vpu_2.cu.covariance)
    r = vpu_1.forward(ternary_input)
    pred = vpu_1.backward(r) 
    sum_preds += pred
    residual = ternary_input - non_linearity(pred) 
    # Check values
    print(
        f"PBT: {ternary_input.T}", 
        f"r: {r}", 
        f"ev: {vpu.pi.eigenvector.T}",
        f"lambda: {vpu.pi.eigenvalue}",
        f"Scaled r: {r*np.sqrt(vpu.pi.eigenvalue)}", 
        f"NL r: {non_linearity(r)}", 
        f"Predictions: {pred.T}", 
        f"NL preds: {non_linearity(pred).T}", 
        f"Residual: {residual.T}",
        sep="\n", end="\n-------\n")

PBT: [[-1  0]]
r: [[-0.95898732]]
ev: [[ 0.95898732 -0.28344897]]
lambda: [[0.30013363]]
Scaled r: [[-0.52537595]]
NL r: [[-0.]]
Predictions: [[-0.91965668  0.27182397]]
NL preds: [[-1.  0.]]
Residual: [[0. 0.]]
-------
PBT: [[0 1]]
r: [[-0.28341299]]
ev: [[ 0.95899795 -0.28341299]]
lambda: [[0.30012012]]
Scaled r: [[-0.15526276]]
NL r: [[-0.]]
Predictions: [[-0.27179248  0.08032292]]
NL preds: [[-0.  0.]]
Residual: [[0. 1.]]
-------
PBT: [[0 1]]
r: [[-0.28334884]]
ev: [[ 0.95901691 -0.28334884]]
lambda: [[0.30010662]]
Scaled r: [[-0.15522413]]
NL r: [[-1.]]
Predictions: [[-0.27173633  0.08028657]]
NL preds: [[-0.  0.]]
Residual: [[0. 1.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.95901525 -0.28335445]]
lambda: [[0.30010357]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
Residual: [[0. 0.]]
-------
PBT: [[-1  1]]
r: [[-1.24242982]]
ev: [[ 0.95899003 -0.28343979]]
lambda: [[0.3001174]]
Scaled r: [[-0.68063998]]
NL r: [[-1.]]
Predictions: [[-1.19147782  0.3521540

PBT: [[-1  0]]
r: [[-0.9581411]]
ev: [[ 0.9581411 -0.2862964]]
lambda: [[0.30005823]]
Scaled r: [[-0.52484642]]
NL r: [[-1.]]
Predictions: [[-0.91803437  0.27431235]]
NL preds: [[-1.  0.]]
Residual: [[ 0. -1.]]
-------
PBT: [[0 1]]
r: [[-0.28621652]]
ev: [[ 0.95816497 -0.28621652]]
lambda: [[0.30004478]]
Scaled r: [[-0.15677895]]
NL r: [[-0.]]
Predictions: [[-0.27424265  0.0819199 ]]
NL preds: [[-0.  0.]]
Residual: [[0. 1.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.95816644 -0.28621158]]
lambda: [[0.30004184]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
Residual: [[0. 0.]]
-------
PBT: [[0 1]]
r: [[-0.28616811]]
ev: [[ 0.95817942 -0.28616811]]
lambda: [[0.30002839]]
Scaled r: [[-0.15674815]]
NL r: [[-0.]]
Predictions: [[-0.2742004   0.08189219]]
NL preds: [[-0.  0.]]
Residual: [[0. 1.]]
-------
PBT: [[-1  0]]
r: [[-0.95821314]]
ev: [[ 0.95821314 -0.28605521]]
lambda: [[0.30002398]]
Scaled r: [[-0.52485593]]
NL r: [[-0.]]
Predictions: [[-0.91817242  0.2741018

lambda: [[0.29988724]]
Scaled r: [[-0.52463562]]
NL r: [[-1.]]
Predictions: [[-0.9178201   0.27463861]]
NL preds: [[-1.  1.]]
Residual: [[0. 0.]]
-------
PBT: [[-1  0]]
r: [[-0.95805724]]
ev: [[ 0.95805724 -0.28657691]]
lambda: [[0.29988281]]
Scaled r: [[-0.52464706]]
NL r: [[-1.]]
Predictions: [[-0.91787367  0.27455709]]
NL preds: [[-1.  0.]]
Residual: [[0. 0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.95806144 -0.28656287]]
lambda: [[0.29987993]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
Residual: [[0. 0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.95805012 -0.28660073]]
lambda: [[0.29987705]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
Residual: [[0. 0.]]
-------
PBT: [[-1  0]]
r: [[-0.95806785]]
ev: [[ 0.95806785 -0.28654145]]
lambda: [[0.29987262]]
Scaled r: [[-0.52464396]]
NL r: [[-1.]]
Predictions: [[-0.917894    0.27452615]]
NL preds: [[-1.  0.]]
Residual: [[0. 0.]]
-------
PBT: [[0 1]]
r: [[-0.28646256]]
ev: [[ 0.9580914

PBT: [[-1  1]]
r: [[-1.2449223]]
ev: [[ 0.95793412 -0.28698818]]
lambda: [[0.29986272]]
Scaled r: [[-0.68171599]]
NL r: [[-1.]]
Predictions: [[-1.19255355  0.35727798]]
NL preds: [[-1.  0.]]
Residual: [[0. 1.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.95790519 -0.28708473]]
lambda: [[0.29985993]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
Residual: [[0. 0.]]
-------
PBT: [[-1  1]]
r: [[-1.24508976]]
ev: [[ 0.95786245 -0.28722732]]
lambda: [[0.29987345]]
Scaled r: [[-0.6818199]]
NL r: [[-1.]]
Predictions: [[-1.19262473  0.35762379]]
NL preds: [[-1.  0.]]
Residual: [[0. 1.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.95782043 -0.28736739]]
lambda: [[0.29987067]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
Residual: [[0. 0.]]
-------
PBT: [[-1  0]]
r: [[-0.95781796]]
ev: [[ 0.95781796 -0.28737564]]
lambda: [[0.29986618]]
Scaled r: [[-0.52450148]]
NL r: [[-1.]]
Predictions: [[-0.91741524  0.27525355]]
NL preds: [[-1.  1.]]
Residual: [[

PBT: [[0 1]]
r: [[-0.28922434]]
ev: [[ 0.95726135 -0.28922434]]
lambda: [[0.2998263]]
Scaled r: [[-0.15836882]]
NL r: [[-0.]]
Predictions: [[-0.27686328  0.08365072]]
NL preds: [[-0.  0.]]
Residual: [[0. 1.]]
-------
PBT: [[-1  1]]
r: [[-1.24650458]]
ev: [[ 0.95725316 -0.28925141]]
lambda: [[0.29983976]]
Scaled r: [[-0.68255632]]
NL r: [[-1.]]
Predictions: [[-1.19322045  0.36055321]]
NL preds: [[-1.  0.]]
Residual: [[0. 1.]]
-------
PBT: [[-1  0]]
r: [[-0.95727283]]
ev: [[ 0.95727283 -0.28918633]]
lambda: [[0.2998352]]
Scaled r: [[-0.52417589]]
NL r: [[-1.]]
Predictions: [[-0.91637127  0.27683022]]
NL preds: [[-1.  0.]]
Residual: [[0. 0.]]
-------
PBT: [[-1  1]]
r: [[-1.24648434]]
ev: [[ 0.95726193 -0.28922241]]
lambda: [[0.29984866]]
Scaled r: [[-0.68255536]]
NL r: [[-1.]]
Predictions: [[-1.193212    0.36051121]]
NL preds: [[-1.  0.]]
Residual: [[0. 1.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.95724068 -0.28929274]]
lambda: [[0.29984596]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0

...

PBT: [[0 1]]
r: [[-0.29297836]]
ev: [[ 0.95611907 -0.29297836]]
lambda: [[0.29979571]]
Scaled r: [[-0.16041621]]
NL r: [[-0.]]
Predictions: [[-0.2801222   0.08583632]]
NL preds: [[-1.  0.]]
Residual: [[0. 1.]]
-------
PBT: [[-1  0]]
r: [[-0.95616571]]
ev: [[ 0.95616571 -0.29282613]]
lambda: [[0.299791]]
Scaled r: [[-0.52353107]]
NL r: [[-1.]]
Predictions: [[-0.91425286  0.2799903 ]]
NL preds: [[-1.  0.]]
Residual: [[0. 0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.95618202 -0.29277286]]
lambda: [[0.29978848]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
Residual: [[0. 0.]]
-------
PBT: [[-1  1]]
r: [[-1.24898424]]
ev: [[ 0.95616906 -0.29281518]]
lambda: [[0.29980178]]
Scaled r: [[-0.6838708]]
NL r: [[-1.]]
Predictions: [[-1.19424008  0.36572154]]
NL preds: [[-1.  0.]]
Residual: [[0. 0.]]
-------
PBT: [[-1  1]]
r: [[-1.24905692]]
ev: [[ 0.95613697 -0.29291995]]
lambda: [[0.29981508]]
Scaled r: [[-0.68392577]]
NL r: [[-1.]]
Predictions: [[-1.1942695   0.3658736

Scaled r: [[-0.15896551]]
NL r: [[-0.]]
Predictions: [[-0.2779591  0.0843815]]
NL preds: [[-0.  0.]]
Residual: [[0. 1.]]
-------
PBT: [[-1  1]]
r: [[-1.24738909]]
ev: [[ 0.95686884 -0.29052025]]
lambda: [[0.29948679]]
Scaled r: [[-0.6826385]]
NL r: [[-1.]]
Predictions: [[-1.19358776  0.36239179]]
NL preds: [[-1.  0.]]
Residual: [[0. 1.]]
-------
PBT: [[-1  0]]
r: [[-0.95688653]]
ev: [[ 0.95688653 -0.29046199]]
lambda: [[0.29948217]]
Scaled r: [[-0.52365581]]
NL r: [[-0.]]
Predictions: [[-0.91563183  0.27793917]]
NL preds: [[-1.  0.]]
Residual: [[ 0. -1.]]
-------
PBT: [[0 1]]
r: [[-0.2903842]]
ev: [[ 0.95691014 -0.2903842 ]]
lambda: [[0.29946917]]
Scaled r: [[-0.1589092]]
NL r: [[-1.]]
Predictions: [[-0.27787159  0.08432298]]
NL preds: [[-0.  1.]]
Residual: [[1. 1.]]
-------
PBT: [[0 1]]
r: [[-0.29029357]]
ev: [[ 0.95693764 -0.29029357]]
lambda: [[0.29945617]]
Scaled r: [[-0.15885616]]
NL r: [[-1.]]
Predictions: [[-0.27779285  0.08427036]]
NL preds: [[-1.  0.]]
Residual: [[1. 1.]]
----

PBT: [[-1  1]]
r: [[-1.24529425]]
ev: [[ 0.9577748  -0.28751945]]
lambda: [[0.29923268]]
Scaled r: [[-0.68120291]]
NL r: [[-1.]]
Predictions: [[-1.19271145  0.35804632]]
NL preds: [[-1.  1.]]
Residual: [[0. 1.]]
-------
PBT: [[0 1]]
r: [[-0.2874745]]
ev: [[ 0.95778829 -0.2874745 ]]
lambda: [[0.29921982]]
Scaled r: [[-0.15725139]]
NL r: [[-1.]]
Predictions: [[-0.27533971  0.08264159]]
NL preds: [[-0.  0.]]
Residual: [[1. 1.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.95778322 -0.28749139]]
lambda: [[0.29921733]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
Residual: [[0. 0.]]
-------
PBT: [[0 1]]
r: [[-0.28746257]]
ev: [[ 0.95779187 -0.28746257]]
lambda: [[0.29920447]]
Scaled r: [[-0.15724083]]
NL r: [[-0.]]
Predictions: [[-0.27532931  0.08263473]]
NL preds: [[-0.  0.]]
Residual: [[0. 1.]]
-------
PBT: [[0 1]]
r: [[-0.28740365]]
ev: [[ 0.95780955 -0.28740365]]
lambda: [[0.29919161]]
Scaled r: [[-0.15720523]]
NL r: [[-0.]]
Predictions: [[-0.27527797  0.08260086]

PBT: [[0 1]]
r: [[-0.2897852]]
ev: [[ 0.95709171 -0.2897852 ]]
lambda: [[0.29919832]]
Scaled r: [[-0.15850967]]
NL r: [[-0.]]
Predictions: [[-0.27735101  0.08397546]]
NL preds: [[-0.  0.]]
Residual: [[1. 1.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.95707717 -0.28983322]]
lambda: [[0.29919593]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
Residual: [[0. 0.]]
-------
PBT: [[0 1]]
r: [[-0.28982521]]
ev: [[ 0.95707959 -0.28982521]]
lambda: [[0.29918312]]
Scaled r: [[-0.15852753]]
NL r: [[-1.]]
Predictions: [[-0.2773858   0.08399865]]
NL preds: [[-0.  0.]]
Residual: [[1. 1.]]
-------
PBT: [[-1  1]]
r: [[-1.24695432]]
ev: [[ 0.95705808 -0.28989624]]
lambda: [[0.29919602]]
Scaled r: [[-0.68206921]]
NL r: [[-1.]]
Predictions: [[-1.19340771  0.36148737]]
NL preds: [[-1.  0.]]
Residual: [[0. 0.]]
-------
PBT: [[-1  0]]
r: [[-0.95706831]]
ev: [[ 0.95706831 -0.28986246]]
lambda: [[0.29919139]]
Scaled r: [[-0.52350096]]
NL r: [[-1.]]
Predictions: [[-0.91597976  0.2774181

PBT: [[0 1]]
r: [[-0.28967681]]
ev: [[ 0.95712452 -0.28967681]]
lambda: [[0.29908381]]
Scaled r: [[-0.15842006]]
NL r: [[-0.]]
Predictions: [[-0.27725678  0.08391265]]
NL preds: [[-0.  0.]]
Residual: [[1. 1.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.95710399 -0.28974464]]
lambda: [[0.29908148]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
Residual: [[0. 0.]]
-------
PBT: [[0 0]]
r: [[0.]]
ev: [[ 0.95707643 -0.28983564]]
lambda: [[0.29907915]]
Scaled r: [[0.]]
NL r: [[0.]]
Predictions: [[0. 0.]]
NL preds: [[0. 0.]]
Residual: [[0. 0.]]
-------
PBT: [[-1  1]]
r: [[-1.24700673]]
ev: [[ 0.9570353  -0.28997143]]
lambda: [[0.29909193]]
Scaled r: [[-0.68197922]]
NL r: [[-1.]]
Predictions: [[-1.19342947  0.36159633]]
NL preds: [[-1.  1.]]
Residual: [[0. 1.]]
-------
PBT: [[0 1]]
r: [[-0.29002135]]
ev: [[ 0.95702017 -0.29002135]]
lambda: [[0.2990792]]
Scaled r: [[-0.15860727]]
NL r: [[-0.]]
Predictions: [[-0.27755629  0.08411239]]
NL preds: [[-0.  0.]]
Residual: [[0. 

In [191]:
print(vpu.cu.mean)

[[-0.27731322]
 [ 0.38556626]]


Because we have a static value the CU mean is biased based on the static value. So we can ignore here - in real life we have variation so this should tend to 0 > need to check. If it is not 0 in real like we would need to constantly adapt such that it is 0.

In [184]:
print(np.sqrt(vpu.pi.eigenvalue))

[[0.54316595]]
